<a href="https://colab.research.google.com/github/AnandaSheva/expert_system/blob/main/plant_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import library
import re
import json
from typing import List, Dict
from google.colab import files
import os

In [ ]:
# Data tanaman
plant_data_text = """Data Tanaman:
1. Nama Tanaman: Bawang Merah, Ketinggian Tempat (mdpl): 0 - 450, Curah Hujan(mm/bulan): 25 - 250, Bulan Tanam: April, Mei, Juli, Agustus, Oktober, November;
2. Nama Tanaman: Bawang Putih, Ketinggian Tempat (mdpl): 700 - 1500, Curah Hujan (mm/bulan): 110 - 200, Bulan Tanam: Mei, Juni, Juli;
3. Nama Tanaman: Cabai, Ketinggian Tempat (mdpl): 0 - 1000, Curah Hujan (mm/bulan): 100 - 200, Bulan Tanam: Juli, Agustus, September, Oktober;
4. Nama Tanaman: Jagung, Ketinggian Tempat (mdpl): 0 - 600, Curah Hujan (mm/bulan): 85 - 200, Bulan Tanam: Maret, Juni, Oktober, Desember;
5. Nama Tanaman: Kedelai, Ketinggian Tempat (mdpl): 0 - 600, Curah Hujan (mm/bulan): 100 - 400, Bulan Tanam: April, Mei, Juni, Juli, Agustus;
6. Nama Tanaman: Kentang, Ketinggian Tempat (mdpl): 500 - 3000, Curah Hujan (mm/bulan): 200 - 300, Bulan Tanam: April, Mei, Oktober, November;
7. Nama Tanaman: Kubis/Kol, Ketinggian Tempat (mdpl): 0 - 1000, Curah Hujan (mm/bulan): 70 - 100, Bulan Tanam: September, Oktober, November;
8. Nama Tanaman: Padi/Beras, Ketinggian Tempat (mdpl): 0 - 1000, Curah Hujan (mm/bulan): 80 - 170, Bulan Tanam: April, Mei, Juni, Oktober, November, Desember;
9. Nama Tanaman: Tomat, Ketinggian Tempat (mdpl): 0 - 1500, Curah Hujan (mm/bulan): 100 - 220, Bulan Tanam: Juni, Juli, Agustus;
10. Nama Tanaman: Wortel, Ketinggian Tempat (mdpl): 800 - 1500, Curah Hujan (mm/bulan): 70 - 100, Bulan Tanam: Oktober, November, Desember, Januari, Februari, Maret;"""

# Ekstrak data
plants_data = extract_plant_data(plant_data_text)

# Simpan data ke variabel global
PLANTS_DATABASE = plants_data

# Tampilkan data dalam format yang mudah dibaca
print("Data Tanaman yang Berhasil Diekstrak:")
print(json.dumps(PLANTS_DATABASE, indent=2, ensure_ascii=False))


Data Tanaman yang Berhasil Diekstrak:
[
  {
    "nama_tanaman": "Bawang Merah",
    "ketinggian_tempat": {
      "min": 0.0,
      "max": 450.0
    },
    "curah_hujan": {
      "min": 25.0,
      "max": 250.0
    },
    "bulan_tanam": [
      "April",
      "Mei",
      "Juli",
      "Agustus",
      "Oktober",
      "November"
    ]
  },
  {
    "nama_tanaman": "Bawang Putih",
    "ketinggian_tempat": {
      "min": 700.0,
      "max": 1500.0
    },
    "curah_hujan": {
      "min": 110.0,
      "max": 200.0
    },
    "bulan_tanam": [
      "Mei",
      "Juni",
      "Juli"
    ]
  },
  {
    "nama_tanaman": "Cabai",
    "ketinggian_tempat": {
      "min": 0.0,
      "max": 1000.0
    },
    "curah_hujan": {
      "min": 100.0,
      "max": 200.0
    },
    "bulan_tanam": [
      "Juli",
      "Agustus",
      "September",
      "Oktober"
    ]
  },
  {
    "nama_tanaman": "Jagung",
    "ketinggian_tempat": {
      "min": 0.0,
      "max": 600.0
    },
    "curah_hujan": {
      "mi

In [ ]:
#Membuat fungsi ekstrasi data tanaman
def extract_plant_data(text: str) -> List[Dict]:
    plants = []
    pattern = r'Nama Tanaman: (.*?), Ketinggian Tempat \(mdpl\): (.*?), Curah\s+Hujan\s*\(mm/bulan\): (.*?), Bulan Tanam: (.*?);'

    matches = re.finditer(pattern, text)

    for match in matches:
        try:
            plant = {
                "nama_tanaman": match.group(1).strip(),
                "ketinggian_tempat": parse_range(match.group(2)),
                "curah_hujan": parse_range(match.group(3)),
                "bulan_tanam": [month.strip() for month in match.group(4).split(',')]
            }
            plants.append(plant)
        except ValueError as e:
            print(f"Error parsing data for plant: {match.group(1)}")
            print(f"Error message: {str(e)}")
            continue

    return plants

In [ ]:
#Fungsi mengurai nilai
def parse_range(range_str: str) -> Dict[str, float]:
    cleaned_str = range_str.strip().replace(" ", "")
    numbers = re.findall(r'\d+', cleaned_str)

    if len(numbers) >= 2:
        return {
            "min": float(numbers[0]),
            "max": float(numbers[1])
        }
    else:
        raise ValueError(f"Format range tidak valid: {range_str}")

#Fungsi mengecek nilai
def is_in_range(value: float, range_dict: Dict[str, float]) -> bool:
    return range_dict["min"] <= value <= range_dict["max"]

#fungsi memberikan rekomendasi tanaman
def recommend_plants(ketinggian: float, curah_hujan: float, bulan: str, plants_data: List[Dict]) -> List[str]:

    recommendations = []

    for plant in plants_data:
        if (is_in_range(ketinggian, plant["ketinggian_tempat"]) and
            is_in_range(curah_hujan, plant["curah_hujan"]) and
            bulan in plant["bulan_tanam"]):
            recommendations.append(plant["nama_tanaman"])

    return recommendations

#Fungsi mendapatkan rekomendasi
def get_plant_recommendation(ketinggian: float, curah_hujan: float, bulan: str) -> List[str]:
    return recommend_plants(ketinggian, curah_hujan, bulan, PLANTS_DATABASE)

In [ ]:
def run_recommendation_system():
    print("Sistem Rekomendasi Tanaman")
    print("==========================")

    try:
        ketinggian = float(input("Masukkan ketinggian tempat (mdpl): "))
        curah_hujan = float(input("Masukkan curah hujan (mm/bulan): "))
        bulan = input("Masukkan bulan tanam (contoh: April): ").capitalize()

        rekomendasi = get_plant_recommendation(ketinggian, curah_hujan, bulan)

        print("\nHasil Rekomendasi:")
        if rekomendasi:
            print(f"Untuk lokasi dengan ketinggian {ketinggian} mdpl, curah hujan {curah_hujan} mm/bulan,")
            print(f"pada bulan {bulan}, tanaman yang direkomendasikan adalah:")
            for i, tanaman in enumerate(rekomendasi, 1):
                print(f"{i}. {tanaman}")
        else:
            print("Tidak ada tanaman yang cocok dengan kriteria tersebut.")

    except ValueError as e:
        print("\nError: Masukkan nilai yang valid!")
        print("Ketinggian dan curah hujan harus berupa angka.")

In [ ]:
#Jalankan sistem rekomendasi
run_recommendation_system()

Sistem Rekomendasi Tanaman
Masukkan ketinggian tempat (mdpl): 250
Masukkan curah hujan (mm/bulan): 100
Masukkan bulan tanam (contoh: April): Desember

Hasil Rekomendasi:
Untuk lokasi dengan ketinggian 250.0 mdpl, curah hujan 100.0 mm/bulan,
pada bulan Desember, tanaman yang direkomendasikan adalah:
1. Jagung
2. Padi/Beras
